## Opening images in Napari viewer

In [ ]:
from pathlib import Path
import tifffile as tif
import numpy as np
import napari

# ------------------------------------------------------------------
# 1.  Fixed directories
# ------------------------------------------------------------------
PROC_DIR = Path(
    "/Users/jeremy/Library/CloudStorage/OneDrive-CRG-CentredeRegulacioGenomica/"
    "Microscopy/2024-07-17_CA1200_JA1880/Processed"
)
MASK_DIR = Path(
    "/Users/jeremy/Library/CloudStorage/OneDrive-CRG-CentredeRegulacioGenomica/"
    "Microscopy/2024-07-17_CA1200_JA1880/Masks"
)

# ------------------------------------------------------------------
# 2.  Pick the image base name
# ------------------------------------------------------------------
image_name = "JA1880_B"

# ------------------------------------------------------------------
# 3.  Resolve file paths
# ------------------------------------------------------------------
proc_path  = PROC_DIR / f"{image_name}_processed.tif"
germ_path  = MASK_DIR / f"{image_name}_germline.tif"
soma_path  = MASK_DIR / f"{image_name}_soma.tif"

for p in (proc_path, germ_path, soma_path):
    if not p.exists():
        raise FileNotFoundError(p)

# ------------------------------------------------------------------
# 4.  Load images
# ------------------------------------------------------------------
stack = tif.imread(proc_path)
red   = stack[0]               # channel 0 (red)
germ  = tif.imread(germ_path)
soma  = tif.imread(soma_path)

# ------------------------------------------------------------------
# 5.  Launch napari (new pattern)
# ------------------------------------------------------------------
viewer = napari.Viewer(title=image_name)

viewer.add_image(
    red,
    name="red",
    colormap="magenta",
    contrast_limits=(np.percentile(red, 1), np.percentile(red, 99)),
)

germ_layer = viewer.add_labels(germ, name="germline", opacity=0.5)
germ_layer.color = {i: "green" for i in range(1, 6)}

soma_layer = viewer.add_labels(soma, name="soma", opacity=0.5)
soma_layer.color = {i: "#ff66ff" for i in range(1, 6)}

# start the event loop *once* at the end of the script
napari.run()

## Quantification


In [ ]:
from pathlib import Path
import tifffile as tif
import numpy as np
import pandas as pd

"""Quantify red‑channel fluorescence in germline, soma, and head masks.

Image naming:  <condition>_<replicate>_processed.tif  (2‑channel, red = ch‑0)
Mask naming :  same base +  _germline.tif / _soma.tif / _head.tif (labels 1‑5)

Per‑image background offsets (after camera upgrade 2024‑07‑17):
    CA1200_A_processed → 10253
    CA1200_B_processed →  8752
    JA1880_A_processed →  9317
    JA1880_B_processed →  8226

The script subtracts the correct background per stack; any unlisted stack
falls back to 0.
Output: quantification_soma_germline_head.csv
"""

# -----------------------------------------------------------------------------
# Paths ------------------------------------------------------------------------
PROC_DIR = Path(
    "/Users/jeremy/Library/CloudStorage/OneDrive-CRG-CentredeRegulacioGenomica/Microscopy/2024-07-17_CA1200_JA1880/Processed"
)
MASK_DIR = Path(
    "/Users/jeremy/Library/CloudStorage/OneDrive-CRG-CentredeRegulacioGenomica/Microscopy/2024-07-17_CA1200_JA1880/Masks"
)

BACKGROUND_MAP = {
    "CA1200_A": 10253,
    "CA1200_B":  8752,
    "JA1880_A":  9317,
    "JA1880_B":  8226,
}
DEFAULT_BACKGROUND = 0

# -----------------------------------------------------------------------------
# Helper -----------------------------------------------------------------------

def mean_in_label(mask: np.ndarray, channel: np.ndarray, label_id: int) -> float:
    px = channel[mask == label_id]
    return float(np.nan) if px.size == 0 else float(px.mean())

# -----------------------------------------------------------------------------
# Main loop --------------------------------------------------------------------
records: list[dict] = []

for proc_path in sorted(PROC_DIR.glob("*_processed.tif")):
    stem = proc_path.stem[:-10]  # drop "_processed"
    try:
        condition, replicate = stem.split("_", 1)
    except ValueError:
        condition, replicate = stem, ""

    germ_path = MASK_DIR / f"{stem}_germline.tif"
    soma_path = MASK_DIR / f"{stem}_soma.tif"
    head_path = MASK_DIR / f"{stem}_head.tif"
    if not (germ_path.exists() and soma_path.exists() and head_path.exists()):
        print(f"⚠️  Missing mask(s) for {stem}; skipping.")
        continue

    # load red channel (channel 0) & subtract per‑image background
    stack = tif.imread(proc_path)
    red = stack[0].astype(np.float32)
    bg  = BACKGROUND_MAP.get(stem, DEFAULT_BACKGROUND)
    red = red - bg
    red[red < 0] = 0

    germ = tif.imread(germ_path)
    soma = tif.imread(soma_path)
    head = tif.imread(head_path)

    for worm in range(1, 6):
        records.append({
            "condition": condition,
            "replicate": replicate,
            "worm": worm,
            "germline_mean": mean_in_label(germ, red, worm),
            "soma_mean":     mean_in_label(soma, red, worm),
            "head_mean":     mean_in_label(head, red, worm),
        })

# -----------------------------------------------------------------------------
# Save CSV ---------------------------------------------------------------------
results = pd.DataFrame(records)
results.to_csv(PROC_DIR / "quantification_soma_germline_head.csv", index=False)
print("✓ Quantification saved →", PROC_DIR / "quantification_soma_germline_head.csv")

## Plotting

In [ ]:

# ------------------------------------------------------------------
# Box-and-whisker: CA1200 vs JA1880 (replicates pooled)
# ------------------------------------------------------------------
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd

# 1.  Load the CSV we just wrote
csv_path = PROC_DIR / "quantification_soma_germline_head.csv"
df = pd.read_csv(csv_path)

# 2.  Keep only the two conditions, pool replicates A+B
df = df[df["condition"].isin(["CA1200", "JA1880"])]

tissue_cols = {
    "germline_mean": "Germline",
    "soma_mean":     "Soma",
    "head_mean":     "Head",
}
colors = {"CA1200": "#5CB2B1", "JA1880": "#D6766C"}  # teal / pale-red

fig, axes = plt.subplots(1, 3, figsize=(10, 4), sharey=True)

for ax, (col, label) in zip(axes, tissue_cols.items()):
    data_ca = df.loc[df["condition"] == "CA1200", col]
    data_ja = df.loc[df["condition"] == "JA1880", col]

    # We draw two separate boxes so each can have its own colour
    b1 = ax.boxplot(
        [data_ca], positions=[1], widths=0.6, patch_artist=True,
        boxprops=dict(facecolor=colors["CA1200"], edgecolor="black"),
        medianprops=dict(color="black"),
        whiskerprops=dict(color="black"), capprops=dict(color="black"),
        flierprops=dict(marker="o", markersize=4, markerfacecolor=colors["CA1200"], markeredgecolor="black")
    )
    b2 = ax.boxplot(
        [data_ja], positions=[2], widths=0.6, patch_artist=True,
        boxprops=dict(facecolor=colors["JA1880"], edgecolor="black"),
        medianprops=dict(color="black"),
        whiskerprops=dict(color="black"), capprops=dict(color="black"),
        flierprops=dict(marker="o", markersize=4, markerfacecolor=colors["JA1880"], markeredgecolor="black")
    )

    ax.set_xticks([1, 2])
    ax.set_xticklabels(["CA1200", "JA1880"])
    ax.set_title(label)
    ax.yaxis.set_major_locator(mticker.MaxNLocator(5))

axes[0].set_ylabel("Mean red intensity (background-subtracted)")
plt.tight_layout()
plt.show()